## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.

In [1]:
#!pip install sqlalchemy==1.3.9

import csv, sqlite3

In [2]:
### Connect to the database

In [3]:
%load_ext sql

In [4]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
%sql sqlite:///my_data1.db

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

D:\Users\akhil\anaconda3\envs\Db_sql_3_7_12\lib\site-packages\pandas\core\generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [7]:
df=pd.read_csv("E:\\Downloads\\Spacex.csv")

In [8]:
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,6/4/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,12/8/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,10/8/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,3/1/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [9]:
# removing black rows from the table

In [10]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [16]:
%sql\
select  * from SPACEXTABLE\
limit 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
6/4/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
12/8/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
10/8/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
3/1/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


Task 3
Display the total payload mass carried by boosters launched by NASA (CRS)

In [17]:
%sql select customer,sum(payload_mass__kg_) as total_mass  from SPACEXTABLE\
where Customer like 'NASA (CRS)'\
group by customer

 * sqlite:///my_data1.db
Done.


Customer,total_mass
NASA (CRS),45596


 Task 4
 Display average payload mass carried by booster version F9 v1.1

In [19]:
%sql select avg(payload_mass__kg_),Booster_Version from\
SPACEXTABLE\
where Booster_Version='F9 v1.1'\
group by Booster_Version

 * sqlite:///my_data1.db
Done.


avg(payload_mass__kg_),Booster_Version
2928.4,F9 v1.1


Task 5
List the date when the first succesful landing outcome in ground pad was acheived.

In [22]:
%sql\
select min(Date) from SPACEXTABLE\
where Landing_Outcome='Success (ground pad)' 

 * sqlite:///my_data1.db
Done.


min(Date)
1/8/2018


Task 6
List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [23]:
%sql\
select booster_version,payload_mass__kg_,Landing_Outcome\
from SPACEXTABLE\
where payload_mass__kg_  between 4000 and 6001 and Landing_Outcome = 'Success (drone ship)'

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_,Landing_Outcome
F9 FT B1022,4696,Success (drone ship)
F9 FT B1026,4600,Success (drone ship)
F9 FT B1021.2,5300,Success (drone ship)
F9 FT B1031.2,5200,Success (drone ship)


Task 7
List the total number of successful and failure mission outcomes

In [25]:
%sql\
select mission_outcome,count(mission_outcome) from SPACEXTABLE\
group by mission_outcome

 * sqlite:///my_data1.db
Done.


Mission_Outcome,count(mission_outcome)
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


Task 8
List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [26]:
%sql\
select booster_version,PAYLOAD_MASS__KG_ from SPACEXTABLE\
where PAYLOAD_MASS__KG_=(select max(PAYLOAD_MASS__KG_) from SPACEXTABLE)

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


Task 9
List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [27]:
%sql\
select date,Booster_Version,Launch_Site,substr(date,-5,-2)as month from SPACEXTABLE\
where Landing_Outcome ='Failure (drone ship)' and date like '%2015%'\
order by date

 * sqlite:///my_data1.db
Done.


Date,Booster_Version,Launch_Site,month
1/10/2015,F9 v1.1 B1012,CCAFS LC-40,10
14/04/2015,F9 v1.1 B1015,CCAFS LC-40,04


 * sqlite:///my_data1.db
Done.


count(landing_outcome),Landing_Outcome
